In [3]:
import os
import pandas as pd

import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import keras
from keras.layers import Dense, Conv2D, MaxPool2D, Dropout, Flatten
from keras.models import Sequential
from keras.preprocessing import image


metadata = pd.read_csv('./metadata.csv')
metadata

,patientid,offset,sex,age,finding,survival,intubated,intubation_present,went_icu,in_icu,...,date,location,folder,filename,doi,url,license,clinical_notes,other_notes,Unnamed: 28
0,2,0.0,M,65.0,COVID-19,Y,NaN,NaN,NaN,NaN,...,"January 22, 2020","Cho Ray Hospital, Ho Chi Minh City, Vietnam",images,auntminnie-a-2020_01_28_23_51_6665_2020_01_28_...,10.1056/nejmc2001272,https://www.nejm.org/doi/full/10.1056/NEJMc200...,NaN,"On January 22, 2020, a 65-year-old man with a ...",NaN,NaN
1,2,3.0,M,65.0,COVID-19,Y,NaN,NaN,NaN,NaN,...,"January 25, 2020","Cho Ray Hospital, Ho Chi Minh City, Vietnam",images,auntminnie-b-2020_01_28_23_51_6665_2020_01_28_...,10.1056/nejmc2001272,https://www.nejm.org/doi/full/10.1056/NEJMc200...,NaN,"On January 22, 2020, a 65-year-old man with a ...",NaN,NaN
2,2,5.0,M,65.0,COVID-19,Y,NaN,NaN,NaN,NaN,...,"January 27, 2020","Cho Ray Hospital, Ho Chi Minh City, Vietnam",images,auntminnie-c-2020_01_28_23_51_6665_2020_01_28_...,10.1056/nejmc2001272,https://www.nejm.org/doi/full/10.1056/NEJMc200...,NaN,"On January 22, 2020, a 65-year-old man with a ...",NaN,NaN
3,2,6.0,M,65.0,COVID-19,Y,NaN,NaN,NaN,NaN,...,"January 28, 2020","Cho Ray Hospital, Ho Chi Minh City, Vietnam",images,auntminnie-d-2020_01_28_23_51_6665_2020_01_28_...,10.1056/nejmc2001272,https://www.nejm.org/doi/full/10.1056/NEJMc200...,NaN,"On January 22, 2020, a 65-year-old man with a ...",NaN,NaN
4,4,0.0,F,52.0,COVID-19,NaN,NaN,NaN,NaN,NaN,...,"January 25, 2020","Changhua Christian Hospital, Changhua City, Ta...",images,nejmc2001573_f1a.jpeg,10.1056/NEJMc2001573,https://www.nejm.org/doi/full/10.1056/NEJMc200...,NaN,diffuse infiltrates in the bilateral lower lungs,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
367,205,11.0,M,55.0,COVID-19,Y,NaN,Y,Y,Y,...,NaN,"North Derbyshire, UK",images,covid-19-pneumonia-progression-and-regression-...,NaN,https://radiopaedia.org/cases/covid-19-pneumon...,CC BY-NC-SA,"ITU admission, Endotracheal tube, nasogastric ...","Case courtesy of Dr Ian Bickle, Radiopaedia.or...",NaN
368,205,13.0,M,55.0,COVID-19,Y,NaN,Y,Y,Y,...,NaN,"North Derbyshire, UK",images,covid-19-pneumonia-progression-and-regression-...,NaN,https://radiopaedia.org/cases/covid-19-pneumon...,CC BY-NC-SA,Lines and tubes suitably sited. Minor regress...,"Case courtesy of Dr Ian Bickle, Radiopaedia.or...",NaN
369,205,20.0,M,55.0,COVID-19,Y,NaN,Y,Y,Y,...,NaN,"North Derbyshire, UK",images,covid-19-pneumonia-progression-and-regression-...,NaN,https://radiopaedia.org/cases/covid-19-pneumon...,CC BY-NC-SA,increasing oxygen requirements. Extubated. Po...,"Case courtesy of Dr Ian Bickle, Radiopaedia.or...",NaN
370,205,24.0,M,55.0,COVID-19,Y,NaN,N,Y,Y,...,NaN,"North Derbyshire, UK",images,covid-19-pneumonia-progression-and-regression-...,NaN,https://radiopaedia.org/cases/covid-19-pneumon...,CC BY-NC-SA,Extubated since the prior radiograph. Partial...,"Case courtesy of Dr Ian Bickle, Radiopaedia.or...",NaN


In [13]:
df = metadata[['offset', 'age', 'sex', 'finding', 'survival', 'view', 'modality', 'clinical_notes']]
df

,offset,age,sex,finding,survival,view,modality,clinical_notes
0,0.0,65.0,M,COVID-19,Y,PA,X-ray,"On January 22, 2020, a 65-year-old man with a ..."
1,3.0,65.0,M,COVID-19,Y,PA,X-ray,"On January 22, 2020, a 65-year-old man with a ..."
2,5.0,65.0,M,COVID-19,Y,PA,X-ray,"On January 22, 2020, a 65-year-old man with a ..."
3,6.0,65.0,M,COVID-19,Y,PA,X-ray,"On January 22, 2020, a 65-year-old man with a ..."
4,0.0,52.0,F,COVID-19,NaN,PA,X-ray,diffuse infiltrates in the bilateral lower lungs
...,...,...,...,...,...,...,...,...
367,11.0,55.0,M,COVID-19,Y,AP Supine,X-ray,"ITU admission, Endotracheal tube, nasogastric ..."
368,13.0,55.0,M,COVID-19,Y,AP Supine,X-ray,Lines and tubes suitably sited. Minor regress...
369,20.0,55.0,M,COVID-19,Y,AP Supine,X-ray,increasing oxygen requirements. Extubated. Po...
370,24.0,55.0,M,COVID-19,Y,AP Supine,X-ray,Extubated since the prior radiograph. Partial...


In [22]:
#Extra Data Analysis 
print(df['view'].value_counts())
print(df['modality'].value_counts())

# PA and AP seem to be the highest. I am not a medical professional but they look similar enough, so both will be utilized.

PA               195
AP                58
AP Supine         45
Axial             41
L                 29
Coronal            3
AP semi erect      1
Name: view, dtype: int64
X-ray    328
CT        44
Name: modality, dtype: int64


In [28]:
#Rename & Filter Function

for count, row in metadata.iterrows():
    src = './images/' + row['filename']
    if (row['view'] == 'PA' or row['view'] == 'AP Supine' or row['view'] == 'AP') and (row['modality'] == 'X-ray'):
        dest = './images_filtered/' + str(count) + '-' + str(df['finding'][count]) + ".jpg"
        os.rename(src, dest)

In [31]:
#Index same conditions as above, check if shape matches 
df = df.loc[((df['view'] == 'PA') | (df['view'] == 'AP Supine') | (df['view'] == 'AP')) & (df['modality'] == 'X-ray')]
df.shape

(298, 8)